# 1. 초기 시작

In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

from pycaret.classification import *

from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import accuracy_score

In [20]:
# 데이터 불러오기
data = pd.read_csv("raw_data.csv", encoding="cp949")

In [21]:
data['INDUSTRY_CD_label'] = data['INDUSTRY_CD'].str[1:].astype('int')

In [22]:
train = data[(data['train']==1)&(data['DATA_CRTR_YM']<=202210)].reset_index()
valid = data[(data['train']==1)&(data['DATA_CRTR_YM']<=202212)&(data['DATA_CRTR_YM']>=202211)].reset_index()
test = data[data['train']==0].reset_index()

In [23]:
train.drop(columns=['전체점포수', '전체 임대료', '65세이상_남', '65세이상_여','index','train'], axis=1, inplace=True)
valid.drop(columns=['전체점포수', '전체 임대료', '65세이상_남', '65세이상_여','index','train'], axis=1, inplace=True)
test.drop(columns=['전체점포수', '전체 임대료', '65세이상_남', '65세이상_여','index','train'], axis=1, inplace=True)

In [24]:
X_train = train[['프랜차이즈점포수', '일반점포수', '길단위유동인구', '주거인구', '직장인구', '개업수', '폐업수', '개업률', '폐업률',
       '1층 임대료', '1층 외 임대료', '생활물가지수', '부동산거래대비유동인구', '공실률대비매매가임대료', '젠트리피케이션',
       '지하철개수', '스타벅스개수', '65세이상']]
X_valid = valid[['프랜차이즈점포수', '일반점포수', '길단위유동인구', '주거인구', '직장인구', '개업수', '폐업수', '개업률', '폐업률',
       '1층 임대료', '1층 외 임대료', '생활물가지수', '부동산거래대비유동인구', '공실률대비매매가임대료', '젠트리피케이션',
       '지하철개수', '스타벅스개수', '65세이상']]
X_test = test[['프랜차이즈점포수', '일반점포수', '길단위유동인구', '주거인구', '직장인구', '개업수', '폐업수', '개업률', '폐업률',
       '1층 임대료', '1층 외 임대료', '생활물가지수', '부동산거래대비유동인구', '공실률대비매매가임대료', '젠트리피케이션',
       '지하철개수', '스타벅스개수', '65세이상']]
y_train = train['SLS_GRD']
y_valid = valid['SLS_GRD']

In [25]:
features_to_scale = X_train.columns
scaler = StandardScaler()
scaler.fit(X_train[features_to_scale])
X_train[features_to_scale] = scaler.transform(X_train[features_to_scale])
X_valid[features_to_scale] = scaler.transform(X_valid[features_to_scale])
X_test[features_to_scale] = scaler.transform(X_test[features_to_scale])

In [26]:
X_train['INDUSTRY_CD_label']= train['INDUSTRY_CD_label']
X_train['대학교여부']= train['대학교여부']
X_valid['INDUSTRY_CD_label']= valid['INDUSTRY_CD_label']
X_valid['대학교여부']= valid['대학교여부']
X_test['INDUSTRY_CD_label']= test['INDUSTRY_CD_label']
X_test['대학교여부']= test['대학교여부']

In [27]:
X_train.head()

,프랜차이즈점포수,일반점포수,길단위유동인구,주거인구,직장인구,개업수,폐업수,개업률,폐업률,1층 임대료,1층 외 임대료,생활물가지수,부동산거래대비유동인구,공실률대비매매가임대료,젠트리피케이션,지하철개수,스타벅스개수,65세이상,INDUSTRY_CD_label,대학교여부
0,-0.708988,-0.325084,-1.258521,-1.359992,-0.472021,-0.242928,-0.464173,-0.461519,-0.603607,0.562311,-0.593268,-1.863554,-2.200829,0.569375,0.243689,-1.032688,-0.098563,-1.077794,3,0
1,0.086300,0.771074,0.722326,0.592735,-0.065615,0.723879,0.218004,0.244111,-0.102444,-0.775482,-0.760337,-1.863554,0.214040,-0.214749,-0.937152,0.833506,-0.098563,-0.319485,1,0
2,0.168597,0.096821,0.722326,0.592735,-0.065615,0.359102,-0.145732,-0.461519,-0.603607,-0.775482,-0.760337,-1.863554,0.139923,-0.687858,-0.578943,0.833506,-0.098563,-0.319485,3,0
3,-0.651786,-0.753883,0.722326,0.592735,-0.065615,-0.758954,-0.605550,-0.840982,1.279872,-0.775482,-0.760337,-1.863554,-1.447554,-0.247792,-0.021729,0.833506,-0.098563,-0.319485,4,0
4,0.086300,0.771074,0.722326,0.592735,-0.065615,0.723879,0.218004,0.244111,-0.102444,-0.775482,-0.760337,-1.863554,0.214040,-0.217753,-0.382886,0.833506,-0.098563,-0.319485,1,0


In [28]:
train_df = pd.concat([X_train, y_train], axis = 1)
valid_df = pd.concat([X_valid, y_valid], axis = 1)

In [34]:
train_df

,프랜차이즈점포수,일반점포수,길단위유동인구,주거인구,직장인구,개업수,폐업수,개업률,폐업률,1층 임대료,...,생활물가지수,부동산거래대비유동인구,공실률대비매매가임대료,젠트리피케이션,지하철개수,스타벅스개수,65세이상,INDUSTRY_CD_label,대학교여부,SLS_GRD
0,-0.708988,-0.325084,-1.258521,-1.359992,-0.472021,-0.242928,-0.464173,-0.461519,-0.603607,0.562311,...,-1.863554,-2.200829,0.569375,0.243689,-1.032688,-0.098563,-1.077794,3,0,4.0
1,0.086300,0.771074,0.722326,0.592735,-0.065615,0.723879,0.218004,0.244111,-0.102444,-0.775482,...,-1.863554,0.214040,-0.214749,-0.937152,0.833506,-0.098563,-0.319485,1,0,5.0
2,0.168597,0.096821,0.722326,0.592735,-0.065615,0.359102,-0.145732,-0.461519,-0.603607,-0.775482,...,-1.863554,0.139923,-0.687858,-0.578943,0.833506,-0.098563,-0.319485,3,0,3.0
3,-0.651786,-0.753883,0.722326,0.592735,-0.065615,-0.758954,-0.605550,-0.840982,1.279872,-0.775482,...,-1.863554,-1.447554,-0.247792,-0.021729,0.833506,-0.098563,-0.319485,4,0,4.0
4,0.086300,0.771074,0.722326,0.592735,-0.065615,0.723879,0.218004,0.244111,-0.102444,-0.775482,...,-1.863554,0.214040,-0.217753,-0.382886,0.833506,-0.098563,-0.319485,1,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961091,-0.651786,-0.601972,-1.149419,-0.508362,-0.617910,-0.758954,-0.742809,-0.840982,-0.808307,-1.410434,...,1.214615,1.889247,-1.356915,-0.087521,-1.032688,-0.098563,0.868043,1,0,4.0
961092,-0.651786,-0.806468,-1.149419,-0.508362,-0.617910,-0.758954,-0.742809,-0.840982,-0.808307,-1.410434,...,1.214615,2.164541,-1.356915,-0.087521,-1.032688,-0.098563,0.868043,7,0,2.0
961093,-0.651786,-0.356577,-0.897562,-0.215883,-0.654383,-0.165821,0.080745,0.722828,1.338694,-1.312876,...,1.214615,1.943701,-0.339410,0.110941,-1.032688,-0.098563,0.395474,1,0,4.0
961094,-0.762499,-0.701299,-0.897562,-0.215883,-0.654383,-0.610670,-0.742809,0.754743,-0.808307,-1.312876,...,1.214615,0.076393,-1.908929,-0.857550,-1.032688,-0.098563,0.395474,6,0,4.0


In [35]:
valid_df

,프랜차이즈점포수,일반점포수,길단위유동인구,주거인구,직장인구,개업수,폐업수,개업률,폐업률,1층 임대료,...,생활물가지수,부동산거래대비유동인구,공실률대비매매가임대료,젠트리피케이션,지하철개수,스타벅스개수,65세이상,INDUSTRY_CD_label,대학교여부,SLS_GRD
0,0.211406,-0.214948,-0.245010,0.317461,-0.221925,-0.070919,-0.225343,-0.507795,-0.560961,-0.286918,...,0.997394,0.368326,0.097125,0.172047,0.833506,-0.098563,0.560934,3,0,2.0
1,-0.467265,-0.788940,-0.245010,0.317461,-0.221925,-0.758954,-0.742809,-0.840982,-0.808307,-0.286918,...,0.997394,0.698073,-1.489890,0.146320,0.833506,-0.098563,0.560934,5,0,2.0
2,0.211406,-0.214948,-0.245010,0.317461,-0.221925,-0.070919,-0.225343,-0.507795,-0.560961,-0.286918,...,0.997394,-1.037638,-0.123131,1.080907,0.833506,-0.098563,0.560934,3,0,1.0
3,0.233917,0.093315,-0.245010,0.317461,-0.221925,-0.017537,-0.468291,0.020709,-0.484786,-0.286918,...,0.997394,-0.617891,0.001529,-0.033522,0.833506,-0.098563,0.560934,1,0,2.0
4,-0.614882,-0.677928,-0.245010,0.317461,-0.221925,-0.758954,-0.468291,-0.840982,1.279872,-0.286918,...,0.997394,-1.857469,-1.458349,0.043132,0.833506,-0.098563,0.560934,12,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194939,0.254215,-0.079396,-0.012423,0.592735,-0.373025,0.058087,0.451344,-0.507795,-0.330673,-0.492602,...,1.156336,1.191937,-0.905638,0.069666,-0.099591,-0.098563,1.382741,3,0,4.0
194940,-0.762499,-0.777254,-0.012423,0.592735,-0.373025,-0.758954,-0.605550,-0.840982,3.397462,-0.492602,...,1.156336,1.688827,-2.798073,0.040184,-0.099591,-0.098563,1.382741,14,0,4.0
194941,-0.356552,-0.619500,-0.012423,0.592735,-0.373025,-0.610670,-0.056514,-0.138863,2.456311,-0.492602,...,1.156336,1.407483,-1.302148,0.849471,-0.099591,-0.098563,1.382741,6,0,3.0
194942,0.254215,-0.079396,-0.012423,0.592735,-0.373025,0.058087,0.451344,-0.507795,-0.330673,-0.492602,...,1.156336,1.424122,-1.850354,-0.425635,-0.099591,-0.098563,1.382741,3,0,4.0


# 2. Pycaret



In [37]:
setC = setup(data = train_df, test_data = valid_df, target = 'SLS_GRD', 
             index = False, categorical_features = ['INDUSTRY_CD_label', '대학교여부'],
             fold = 5, normalize = False, session_id = 42)

,Description,Value
0,Session id,42
1,Target,SLS_GRD
2,Target type,Multiclass
3,Target mapping,"1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4"
4,Original data shape,"(1156040, 21)"
5,Transformed data shape,"(1156040, 35)"
6,Transformed train set shape,"(961096, 35)"
7,Transformed test set shape,"(194944, 35)"
8,Ordinal features,1
9,Numeric features,18


In [38]:
top10 = compare_models(sort = 'AUC', n_select = 10)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.6145,0.8258,0.6128,0.6118,0.6101,0.4916,0.4921,111.1500
knn,K Neighbors Classifier,0.4598,0.7305,0.4578,0.4634,0.4514,0.2867,0.2888,290.9060
dt,Decision Tree Classifier,0.5129,0.6819,0.5110,0.5116,0.5112,0.3638,0.3638,4.7660
lightgbm,Light Gradient Boosting Machine,0.3769,0.6656,0.3871,0.3893,0.3641,0.1543,0.1573,10.3820
gbc,Gradient Boosting Classifier,0.3582,0.6416,0.3693,0.3771,0.3414,0.1235,0.1274,264.9640
ada,Ada Boost Classifier,0.3447,0.6208,0.3566,0.3623,0.3217,0.1020,0.1055,18.5280
lr,Logistic Regression,0.3367,0.6122,0.3496,0.3280,0.3105,0.0867,0.0911,7.7340
lda,Linear Discriminant Analysis,0.3332,0.6108,0.3419,0.3017,0.3072,0.0869,0.0919,3.3100
nb,Naive Bayes,0.2692,0.5806,0.2712,0.2845,0.2271,0.0631,0.0720,1.9480
qda,Quadratic Discriminant Analysis,0.2625,0.5731,0.2652,0.2888,0.2350,0.0629,0.0692,3.3160
